In [3]:
# Import Data
import pandas as pd

data = pd.read_csv("./sp500.csv")
data = data.set_index("Date")

data

,Date,Open,High,Low,Close,5d,10d,20d
0,2014-07-14,0.037816,0.036900,0.044390,0.041064,0.033454,0.027488,0.022811
1,2014-07-15,0.039894,0.037642,0.043134,0.040004,0.033993,0.027862,0.023326
2,2014-07-16,0.039614,0.038036,0.046004,0.042304,0.034485,0.028098,0.023901
3,2014-07-17,0.040557,0.037442,0.040425,0.035798,0.034115,0.027625,0.023918
4,2014-07-18,0.035510,0.036917,0.041878,0.041374,0.034715,0.027419,0.024190
...,...,...,...,...,...,...,...,...
2494,2024-06-10,0.972091,0.977374,0.978436,0.979762,0.982198,0.983501,0.991455
2495,2024-06-11,0.975356,0.980196,0.977249,0.983793,0.986923,0.985486,0.993690
2496,2024-06-12,0.990910,1.000000,1.000000,0.996474,0.990693,0.989902,0.996223
2497,2024-06-13,1.000000,0.998522,0.998161,1.000000,0.995239,0.995584,0.998047
